# 如何将跨线程持久性添加到图表中

<div class="警告提示">
<p class="admonition-title">先决条件</p>
<p>
本指南假设您熟悉以下内容：
<ul>
<li>
<a href="https://langchain-ai.github.io/langgraphjs/concepts/persistence/">
坚持
</a>
</li>
<li>
<a href="https://langchain-ai.github.io/langgraphjs/concepts/memory/">
记忆
</a>
</li>
<li>
<a href="https://js.langchain.com/docs/concepts/#chat-models">
聊天模型
</a>
</li>
</ul>
</p>
</div>

在[上一篇指南](https://langchain-ai.github.io/langgraphjs/how-tos/persistence.ipynb)中，您学习了如何在单个[线程]()上跨多个交互持久保存图形状态。LangGraph.js 还允许您跨**多个线程**保存数据。例如，您可以将有关用户的信息（他们的姓名或首选项）存储在共享内存中，并在新的对话线程中重用它们。

在本指南中，我们将展示如何构建和使用具有使用 [Store](https://langchain-ai.github.io/langgraphjs/reference/classes/checkpoint.BaseStore.html) 接口实现的共享内存的图。

<div class="警告说明">
<p class="admonition-title">注意</p>
<p>
LangGraph.js <code>v0.2.10</code> 中添加了对本指南中使用的 <code><a href="https://langchain-ai.github.io/langgraphjs/reference/classes/checkpoint.BaseStore.html">Store</a></code> API 的支持。
</p>
</div>

## 设置

首先，让我们安装所需的软件包并设置 API 密钥。

<div class="警告提示">
<p class="admonition-title">设置 <a href="https://smith.langchain.com">LangSmith</a> 进行 LangGraph 开发</p>
<p style="padding-top: 5px;">
注册 LangSmith 以快速发现问题并提高 LangGraph 项目的性能。LangSmith 允许您使用跟踪数据来调试、测试和监控使用 LangGraph 构建的 LLM 应用程序 - 在<a href="https://docs.smith.langchain.com">此处</a>了解有关如何开始的更多信息。
</p>
</div>

In [ ]:
// process.env.OPENAI_API_KEY = "sk_...";

// 可选，在 LangSmith 中添加跟踪
// process.env.LANGCHAIN_API_KEY = "lsv2__...";
// process.env.ANTHROPIC_API_KEY = "你的 API 密钥";
// process.env.LANGCHAIN_TRACING_V2 = "true";
// process.env.LANGCHAIN_PROJECT = "跨线程持久化：LangGraphJS";

## 定义商店

在此示例中，我们将创建一个能够检索有关用户首选项的信息的图表。我们将通过定义一个 `InMemoryStore` 来实现这一点 - 一个可以在内存中存储数据并查询该数据的对象。然后，我们将在编译图表时传递 store 对象。这允许图中的每个节点访问存储：当您定义节点函数时，您可以定义 `store` 关键字参数，LangGraph 将自动传递您编译图时使用的存储对象。

使用 `Store` 接口存储对象时，您定义两件事：

* 对象的命名空间，一个元组（类似于目录）
* 对象键（类似于文件名）

在我们的示例中，我们将使用 `("memories", <userId>)` 作为命名空间，并使用随机 UUID 作为每个新内存的密钥。

重要的是，为了确定用户，我们将通过节点函数的 config 关键字参数传递 `userId`。

我们首先定义一个 `InMemoryStore`，它已经填充了有关用户的一些记忆。

In [1]:
import { InMemoryStore } from "@langchain/langgraph";

const inMemoryStore = new InMemoryStore();

## 创建图表

In [ ]:
import { v4 as uuidv4 } from "uuid";
import { ChatAnthropic } from "@langchain/anthropic";
import { BaseMessage } from "@langchain/core/messages";
import {
  Annotation,
  StateGraph,
  START,
  MemorySaver,
  LangGraphRunnableConfig,
  messagesStateReducer,
} from "@langchain/langgraph";

const StateAnnotation = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: messagesStateReducer,
    default: () => [],
  }),
});

const model = new ChatAnthropic({ modelName: "claude-3-5-sonnet-20240620" });

// 注意：我们将 Store 参数传递给节点——
// 这是我们用来编译图表的 Store
const callModel = async (
  state: typeof StateAnnotation.State,
  config: LangGraphRunnableConfig
): Promise<{ messages: any }> => {
  const store = config.store;
  if (!store) {
    if (!store) {
      throw new Error("store is required when compiling the graph");
    }
  }
  if (!config.configurable?.userId) {
    throw new Error("userId is required in the config");
  }
  const namespace = ["memories", config.configurable?.userId];
  const memories = await store.search(namespace);
  const info = memories.map((d) => d.value.data).join("\n");
  const systemMsg = `You are a helpful assistant talking to the user. User info: ${info}`;

  // 如果用户要求模型记住，则存储新的记忆
  const lastMessage = state.messages[state.messages.length - 1];
  if (
    typeof lastMessage.content === "string" &&
    lastMessage.content.toLowerCase().includes("remember")
  ) {
    await store.put(namespace, uuidv4(), { data: lastMessage.content });
  }

  const response = await model.invoke([
    { type: "system", content: systemMsg },
    ...state.messages,
  ]);
  return { messages: response };
};

const builder = new StateGraph(StateAnnotation)
  .addNode("call_model", callModel)
  .addEdge(START, "call_model");

// 注意：编译图表时我们在此处传递存储对象
const graph = builder.compile({
  checkpointer: new MemorySaver(),
  store: inMemoryStore,
});
// 如果您使用 LangGraph Cloud 或 LangGraph Studio，则在编译图表时无需传递存储或检查点，因为它是自动完成的。


<div class="警告提示">
<p class="admonition-title">注意</p>
<p>
如果您使用 LangGraph Cloud 或 LangGraph Studio，则在编译图表时<strong>不需要</strong>传递 store，因为它是自动完成的。
</p>
</div>

## 运行图表！

现在让我们在配置中指定一个用户 ID 并告诉模型我们的名字：

In [3]:
let config = { configurable: { thread_id: "1", userId: "1" } };
let inputMessage = { type: "user", content: "Hi! Remember: my name is Bob" };

for await (const chunk of await graph.stream(
  { messages: [inputMessage] },
  { ...config, streamMode: "values" }
)) {
  console.log(chunk.messages[chunk.messages.length - 1]);
}


HumanMessage {
  "id": "ef28a40a-fd75-4478-929a-5413f2a6b044",
  "content": "Hi! Remember: my name is Bob",
  "additional_kwargs": {},
  "response_metadata": {}
}
AIMessage {
  "id": "msg_01UcHJnSAuVDFuDmqaYkxWAf",
  "content": "Hello Bob! It's nice to meet you. I'll remember that your name is Bob. How can I assist you today?",
  "additional_kwargs": {
    "id": "msg_01UcHJnSAuVDFuDmqaYkxWAf",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-5-sonnet-20240620",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 28,
      "output_tokens": 29
    }
  },
  "response_metadata": {
    "id": "msg_01UcHJnSAuVDFuDmqaYkxWAf",
    "model": "claude-3-5-sonnet-20240620",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 28,
      "output_tokens": 29
    },
    "type": "message",
    "role": "assistant"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_t

In [4]:
config = { configurable: { thread_id: "2", userId: "1" } };
inputMessage = { type: "user", content: "what is my name?" };

for await (const chunk of await graph.stream(
  { messages: [inputMessage] },
  { ...config, streamMode: "values" }
)) {
  console.log(chunk.messages[chunk.messages.length - 1]);
}

HumanMessage {
  "id": "eaaa4e1c-1560-4b0a-9c2d-396313cb000c",
  "content": "what is my name?",
  "additional_kwargs": {},
  "response_metadata": {}
}
AIMessage {
  "id": "msg_01VfqUerYCND1JuWGvbnAacP",
  "content": "Your name is Bob. It's nice to meet you, Bob!",
  "additional_kwargs": {
    "id": "msg_01VfqUerYCND1JuWGvbnAacP",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-5-sonnet-20240620",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 33,
      "output_tokens": 17
    }
  },
  "response_metadata": {
    "id": "msg_01VfqUerYCND1JuWGvbnAacP",
    "model": "claude-3-5-sonnet-20240620",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 33,
      "output_tokens": 17
    },
    "type": "message",
    "role": "assistant"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 33,
    "output_tokens": 17,
    "total_tokens": 50
  }
}

我们现在可以检查内存存储并验证我们实际上已经为用户保存了内存：

In [5]:
const memories = await inMemoryStore.search(["memories", "1"]);
for (const memory of memories) {
    console.log(await memory.value);
}

{ data: 'Hi! Remember: my name is Bob' }


现在让我们为另一个用户运行该图，以验证有关第一个用户的记忆是否是独立的：

In [6]:
config = { configurable: { thread_id: "3", userId: "2" } };
inputMessage = { type: "user", content: "what is my name?" };

for await (const chunk of await graph.stream(
  { messages: [inputMessage] },
  { ...config, streamMode: "values" }
)) {
  console.log(chunk.messages[chunk.messages.length - 1]);
}

HumanMessage {
  "id": "1006b149-de8d-4d8e-81f4-c78c51a7144b",
  "content": "what is my name?",
  "additional_kwargs": {},
  "response_metadata": {}
}
AIMessage {
  "id": "msg_01MjpYZ65NjwZMYq42BWa2Ze",
  "content": "I apologize, but I don't have any information about your name or personal details. As an AI assistant, I don't have access to personal information about individual users unless it's specifically provided in our conversation. Is there something else I can help you with?",
  "additional_kwargs": {
    "id": "msg_01MjpYZ65NjwZMYq42BWa2Ze",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-5-sonnet-20240620",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 25,
      "output_tokens": 56
    }
  },
  "response_metadata": {
    "id": "msg_01MjpYZ65NjwZMYq42BWa2Ze",
    "model": "claude-3-5-sonnet-20240620",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 25,
      "o